In [1]:
from selenium import webdriver as wd
from selenium.webdriver import ActionChains as ac
from time import sleep
import pandas as pd
import re
from bs4 import BeautifulSoup as bs
import math
from selenium.webdriver.common.by import By


In [2]:
def search_keyword(driver, spot):
    search_box = driver.find_element_by_id("search.keyword.query")
    search_box.send_keys(spot)
    driver.find_element_by_css_selector("div.inner_coach_layer").click()
    driver.find_element_by_id("search.keyword.submit").click()

In [3]:
def show_more_info(driver):
    driver.find_element_by_id('info.search.place.more').click()

In [4]:
def pagination1(driver, dumb):
    if dumb == 0:
        return True
    else:
        buttonWrap = driver.find_element_by_xpath('//*[@id="info.search.page"]/div')
        button_num = len(buttonWrap.find_elements_by_css_selector('a'))
        now_page = buttonWrap.find_element_by_css_selector("a.ACTIVE").text
        print(now_page)
        now_page = int(now_page)%5
        if((now_page == 0) | (now_page == button_num)):
            try:
                sleep(1)
                next_button = driver.find_element_by_id('info.search.page.next')
                if(next_button.get_attribute('class') == 'next disabled'):
                    print('last page')
                    return False
                else:
                    next_button.click()
            except:
                print('end')
                return False
        else:
            next_page=now_page+1
            driver.find_element_by_id('info.search.page.no'+str(next_page)).click()

In [16]:
def get_infos(driver, place_category):
    place_df = pd.DataFrame(columns={'p_id','p_name','address','tags', 'open_time', 'open_day', 'category', 'price', 'p_rate', 'link'})
    sleep(2)
    dumb= 0
    try:
        show_more_info(driver)
        sleep(3)
    except:
        print('show_more_info 호출 실패')
        pass

    page = 1
    while (True):
        page_box = driver.find_element_by_css_selector("div.pageWrap")
        pages = page_box.find_elements_by_css_selector("a")
        info_ls = driver.find_elements_by_css_selector("li.PlaceItem.clickArea")
        html = driver.page_source
        soup = bs(html, 'html.parser')
        info_ls = soup.body.find_all('a', class_='numberofscore')
        
        for i in range (0, len(info_ls)):
            print(len(info_ls),'개 결과 중 ', i,' 번째 결과')
            res = info_ls[i].string
            print(res)
            review_count = int(res.split('건')[0])
            if(review_count>0):
                link=info_ls[i]['href']
                # Open a new window
                driver.execute_script("window.open('');")
                # Switch to the new window
                driver.switch_to.window(driver.window_handles[1])
                driver.get(link)
                sleep(1)
                place_df= place_df.append(get_info(driver, place_category, page, i, link))
                driver.close()
                driver.switch_to_window(driver.window_handles[0])
                sleep(1)
                
        page = page + 1
                
        buttonWrap = driver.find_element_by_xpath('//*[@id="info.search.page"]/div')
        button_num = len(buttonWrap.find_elements_by_css_selector('a'))
        now_page = buttonWrap.find_element_by_css_selector("a.ACTIVE").text
        print(now_page)
        now_page = int(now_page)%5
        
        if((now_page == 0) | (now_page == button_num)):
            try:
                sleep(1)
                next_button = driver.find_element_by_id('info.search.page.next')
                if(next_button.get_attribute('class') == 'next disabled'):
                    print('last page')
                    break;
                else:
                    next_button.click()
            except:
                print('end')
                break;
        else:
            try:
                next_page=now_page+1
                driver.find_element_by_id('info.search.page.no'+str(next_page)).click()
                sleep(1)
            except:
                break;
        
    place_df.to_csv(f'./kakao_places_{place_category}.csv', index = False, header=True)

In [6]:
def get_info(driver, place_category, page, index, link):
    place_df= pd.DataFrame(columns={'p_id','p_name','address','tags', 'open_time', 'open_day', 'category', 'price', 'p_rate', 'link'})
    try: 
        place_id= place_category+str(page)+str(index)
    except:
        print('get_info 함수 내 id 할당 오류')
        place_id= None
    try: 
        place_link= link
    except:
        print('get_info 함수 내 link 할당 오류')
        place_link= None
    try: 
        name= driver.find_element_by_css_selector('h2.tit_location').text
    except:
        print('get_info 함수 내 name 할당 오류')
        name= None
    try: 
        open_day= driver.find_element_by_css_selector('ul.list_operation').text
    except:
        print('get_info 함수 내 open_day 할당 오류')
        open_day= None
    try: 
        open_time= driver.find_element_by_css_selector('span.txt_location').text
    except:
        print('get_info 함수 내 open_time 할당 오류')
        open_time= None
    try: 
        tags= driver.find_element_by_css_selector('p.txt_introduce').text
    except:
        print('get_info 함수 내 tags 할당 오류')
        tags= None
    try: 
        address= driver.find_element_by_css_selector('span.txt_address').text
    except:
        print('get_info 함수 내 address 할당 오류')
        address= None
    try: 
        rate= driver.find_element_by_css_selector('em.num_rate').text
    except:
        print('get_info 함수 내 rate 할당 오류')
        rate= None
        
    place_df= place_df.append({
        'p_id': place_id,
        'p_name': name,
        'address': address,
        'tags': tags,
        'open_day': open_day,
        'open_time': open_time,
        'category': place_category,
#         'price': price,
        'p_rate': rate,
        'link': place_link
    }, ignore_index=True)
    
    get_review(driver, place_id)
        
    return place_df

In [7]:
def get_review(driver, place_id):
    review_df=pd.DataFrame(columns={'p_id','u_id','u_rate','review'})
    sleep(2)
    review_count = driver.find_element_by_css_selector('a.tit_subject')
    review_count = int(review_count.find_element_by_css_selector('span.color_b').text)
    all_review_page = math.ceil(review_count/5)
    now_page = 1;
    
    while(now_page <= all_review_page):
        sleep(2)
        review_box = driver.find_element_by_css_selector('ul.list_evaluation')
        review_ls = review_box.find_elements_by_css_selector('li')
        
        for review in review_ls:
            try:
                review.find_element_by_css_selector('button.btn_fold').click()
            except:
                pass
            try:
                score= int(review.find_element_by_css_selector('em.num_rate').text)
            except:
                print('get_review 함수 내 score 할당 오류')
                score= 0
            try: 
                comment= review.find_element_by_css_selector('p.txt_comment').text
            except:
                print('get_review 함수 내 review 할당 오류')
                comment= ''
            try:
                user_id= review.find_element_by_css_selector('a.link_user').text
            except:
                print('get_review 함수 내 user_id 할당 오류')
                user_id= ''
                
            review_df= review_df.append({
                'p_id': place_id,
                'u_id': user_id,
                'u_rate': score,
                'review': comment
            }, ignore_index=True)
            
        
        print('총 '+ str(all_review_page)+'페이지 중, '+ str(now_page) +'페이지 출력 완료')

        if all_review_page == 1:
            break;
            
        sleep(2)
        mapp = driver.find_element_by_css_selector('div.paging_mapdetail')
        page_boxes = mapp.find_elements_by_css_selector("*")
        
        p_p_5 = math.ceil(now_page%5)
        
        try:
            driver.find_element_by_css_selector('a.btn_prev')
            page_boxes[p_p_5+3].click()
            sleep(2)
        except:
            try: 
                page_boxes[p_p_5+2].click()
                sleep(2)
            except:
                try:
                    driver.find_element_by_css_selector('a.btn_next').click()
                    print('다음 페이지')
                    sleep(2)
                except:
                    print('페이지 출력 완료')
                    break;
            
        now_page = now_page + 1;
        
    review_df.to_csv(f'./kakao_reviews/{place_id}', index = False, header=True)

In [24]:
def main():
    region='서울'
    spot_ls = ['볼링장', '공원', '만화카페', '멀티방', '방탈출', '보드카페', '코인노래방']
    spot_kind = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    place_df = pd.DataFrame(columns={'p_id','p_name','address','tags', 'open_time', 'open_day', 'category', 'price', 'p_rate', 'link'})
    for i in range (1, 2):
        url = "https://map.kakao.com/"
        chrome_options=wd.ChromeOptions()
        chrome_options.add_argument('headless')
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('lang=ko_KR')
        driver = wd.Chrome('../../../../usr/local/bin/chromedriver', chrome_options=chrome_options)
        driver.get(url)
        search_keyword(driver, region+' '+spot_ls[i])
        get_infos(driver, spot_kind[i])
        driver.close()
        sleep(10)

In [25]:
main()

/Users/ryeonju/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


15 개 결과 중  0  번째 결과
113건
총 23페이지 중, 1페이지 출력 완료
총 23페이지 중, 2페이지 출력 완료
총 23페이지 중, 3페이지 출력 완료
총 23페이지 중, 4페이지 출력 완료
총 23페이지 중, 5페이지 출력 완료
총 23페이지 중, 6페이지 출력 완료
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
총 23페이지 중, 7페이지 출력 완료
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
총 23페이지 중, 8페이지 출력 완료
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
총 23페이지 중, 9페이지 출력 완료
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
총 23페이지 중, 10페이지 출력 완료
get_review 함수 내 user

/Users/ryeonju/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: use driver.switch_to.window instead


15 개 결과 중  1  번째 결과
47건
get_info 함수 내 open_day 할당 오류
get_info 함수 내 tags 할당 오류
총 10페이지 중, 1페이지 출력 완료
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
총 10페이지 중, 2페이지 출력 완료
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
총 10페이지 중, 3페이지 출력 완료
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
총 10페이지 중, 4페이지 출력 완료
get_review 함수 내 scor

get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
총 23페이지 중, 11페이지 출력 완료
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
총 23페이지 중, 12페이지 출력 완료
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
총 23페이지 중, 13페이지 출력 완료
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
총 23페이지 중, 14페이지 출력 완료
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 

...

페이지 출력 완료
15 개 결과 중  4  번째 결과
51건
총 11페이지 중, 1페이지 출력 완료
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
총 11페이지 중, 2페이지 출력 완료
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
총 11페이지 중, 3페이지 출력 완료
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
총 11페이지 중, 4페이지 출력 완료
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
총 11페이지 중, 5페이지 출력 완료
get_review 함수 내 score 할당 오류
get_revi

get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
총 3페이지 중, 1페이지 출력 완료
get_review 함수 내 user_id 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
총 3페이지 중, 2페이지 출력 완료
페이지 출력 완료
15 개 결과 중  4  번째 결과
2건
총 1페이지 중, 1페이지 출력 완료
15 개 결과 중  5  번째 결과
8건
get_info 함수 내 tags 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
총 2페이지 중, 1페이지 출력 완료
페이지 출력 완료
15 개 결과 중  6  번째 결과
2건
get_info 함수 내 open_day 할당 오류
get_info 함수 내 tags 할당 오류
총 1페이지 중, 1페이지 출력 완료
15 개 결과 중  7  번째 결과
13건
get_info 함수 내 open_day 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
총 3페이지 중, 1페이지 출력 완료
총 3페이지 중,

get_info 함수 내 open_day 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
총 1페이지 중, 1페이지 출력 완료
15 개 결과 중  6  번째 결과
0건
15 개 결과 중  7  번째 결과
0건
15 개 결과 중  8  번째 결과
1건
총 1페이지 중, 1페이지 출력 완료
15 개 결과 중  9  번째 결과
1건
get_info 함수 내 open_day 할당 오류
get_info 함수 내 tags 할당 오류
총 1페이지 중, 1페이지 출력 완료
15 개 결과 중  10  번째 결과
0건
15 개 결과 중  11  번째 결과
0건
15 개 결과 중  12  번째 결과
0건
15 개 결과 중  13  번째 결과
1건
get_info 함수 내 open_day 할당 오류
get_info 함수 내 tags 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
총 1페이지 중, 1페이지 출력 완료
15 개 결과 중  14  번째 결과
0건
7
15 개 결과 중  0  번째 결과
2건
get_info 함수 내 open_day 할당 오류
get_info 함수 내 tags 할당 오류
get_review 함수 내 user_id 할당 오류
총 1페이지 중, 1페이지 출력 완료
15 개 결과 중  1  번째 결과
0건
15 개 결과 중  2  번째 결과
0건
15 개 결과 중  3  번째 결과
2건
get_info 함수 내 open_day 할당 오류
get_info 함수 내 tags 할당 오류
총 1페이지 중, 1페이지 출력 완료
15 개 결과 중  4  번째 결과
7건
get_info 함수 내 open_day 할당 오류
총 2페이지 중, 1페이지 출력 완료
페이지 출력 완료
15 개 결과 중  5  번째 결과
0건
15 개 결과 중  6 

페이지 출력 완료
15 개 결과 중  1  번째 결과
1건
get_info 함수 내 open_day 할당 오류
get_info 함수 내 tags 할당 오류
총 1페이지 중, 1페이지 출력 완료
15 개 결과 중  2  번째 결과
0건
15 개 결과 중  3  번째 결과
3건
get_info 함수 내 open_day 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
총 1페이지 중, 1페이지 출력 완료
15 개 결과 중  4  번째 결과
0건
15 개 결과 중  5  번째 결과
1건
get_info 함수 내 open_day 할당 오류
get_info 함수 내 tags 할당 오류
총 1페이지 중, 1페이지 출력 완료
15 개 결과 중  6  번째 결과
0건
15 개 결과 중  7  번째 결과
0건
15 개 결과 중  8  번째 결과
6건
get_info 함수 내 open_day 할당 오류
get_info 함수 내 tags 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
get_review 함수 내 score 할당 오류
get_review 함수 내 user_id 할당 오류
총 2페이지 중, 1페이지 출력 완료
페이지 출력 완료
15 개 결과 중  9  번째 결과
1건
get_info 함수 내 open_day 할당 오류
get_info 함수 내 tags 할당 오류
총 1페이지 중, 1페이지 출력 완료
15 개 결과 중  10  번째 결과
1건
get_info 함수 내 open_day 할당 오류
get_info 함수 내 tags 할당 

get_info 함수 내 open_day 할당 오류
get_info 함수 내 tags 할당 오류
총 1페이지 중, 1페이지 출력 완료
15 개 결과 중  11  번째 결과
0건
15 개 결과 중  12  번째 결과
0건
15 개 결과 중  13  번째 결과
0건
15 개 결과 중  14  번째 결과
2건
get_info 함수 내 open_day 할당 오류
get_info 함수 내 tags 할당 오류
get_review 함수 내 user_id 할당 오류
총 1페이지 중, 1페이지 출력 완료
24
15 개 결과 중  0  번째 결과
0건
15 개 결과 중  1  번째 결과
0건
15 개 결과 중  2  번째 결과
0건
15 개 결과 중  3  번째 결과
1건
get_info 함수 내 open_day 할당 오류
총 1페이지 중, 1페이지 출력 완료
15 개 결과 중  4  번째 결과
0건
15 개 결과 중  5  번째 결과
0건
15 개 결과 중  6  번째 결과
0건
15 개 결과 중  7  번째 결과
1건
get_info 함수 내 open_day 할당 오류
get_info 함수 내 tags 할당 오류
총 1페이지 중, 1페이지 출력 완료
15 개 결과 중  8  번째 결과
2건
get_info 함수 내 open_day 할당 오류
get_info 함수 내 tags 할당 오류
총 1페이지 중, 1페이지 출력 완료
15 개 결과 중  9  번째 결과
0건
15 개 결과 중  10  번째 결과
0건
15 개 결과 중  11  번째 결과
0건
15 개 결과 중  12  번째 결과
0건
15 개 결과 중  13  번째 결과
0건
15 개 결과 중  14  번째 결과
0건
25
15 개 결과 중  0  번째 결과
0건
15 개 결과 중  1  번째 결과
0건
15 개 결과 중  2  번째 결과
0건
15 개 결과 중  3  번째 결과
1건
get_info 함수 내 open_day 할당 오류
총 1페이지 중, 1페이지 출력 완료
15 개 결과 중  4  번째 결과
0건
15 